In [94]:
import pandas as pd

from watcher import Watcher, Chain
from constants import FUCK_ADDRESS

watcher = Watcher(chain=Chain.SOL)

In [96]:
watcher = Watcher(chain=Chain.SOL)
tokens_invested_response = watcher.grab_tokens_invested(FUCK_ADDRESS)
df = pd.DataFrame(tokens_invested_response.data["holdings"])

In [3]:
import pandas as pd

df = pd.read_csv("./tokens_invested_all.csv")
df['last_active_timestamp']

0      1718132509
1      1718131898
2      1718131538
3      1718131445
4      1718130460
          ...    
638    1717168262
639    1717167983
640    1717165266
641    1717159359
642    1717092654
Name: last_active_timestamp, Length: 643, dtype: int64

In [97]:
df.columns

Index(['address', 'token_address', 'symbol', 'name', 'decimals', 'logo',
       'balance', 'usd_value', 'realized_profit_30d', 'realized_profit',
       'realized_pnl', 'realized_pnl_30d', 'unrealized_profit',
       'unrealized_pnl', 'total_profit', 'total_profit_pnl', 'avg_cost',
       'avg_sold', 'buy_30d', 'sell_30d', 'sells', 'price', 'cost',
       'position_percent', 'last_active_timestamp', 'history_sold_income',
       'history_bought_cost', 'price_change_5m', 'price_change_1h',
       'price_change_6h', 'price_change_24h', 'is_following', 'is_show_alert',
       'is_honeypot'],
      dtype='object')

In [98]:
df[[
    "token_address",
    "symbol",
    "name",
    "realized_profit",
    "cost",
    "history_bought_cost",
    "history_sold_income",
    "total_profit_pnl"
]].head()

,token_address,symbol,name,realized_profit,cost,history_bought_cost,history_sold_income,total_profit_pnl
0,52ScRbUR7y8AVqkMjE29FJVjDPELSPofJwaXaLHepump,TROLL,Troll by Matt Furie,-3.275838,0.743575,7.43575,3.416337,-0.534728
1,8CUbzzN5Q9Tnww8ciDTdcCPXVmi8mymAApot9Hyspump,gloop,Gloop By Matt Furie,-3.778678,0.743175,7.43175,2.909897,-0.602920
2,Fabws3AB7BAowYDywX8Gnb8QBF13TSpSpv3FHUS6pump,SHADES,SHADECREEPER BY MATT FURIE,-3.398939,0.742525,7.42525,3.283786,-0.553187
3,FLJy5tBAxHnzAC3bodjRAtSxoEGmwniqbgQ9LE7Ppump,BILZ,Dan Bilzerian,-4.386256,0.743525,7.43525,2.305469,-0.686733
4,84mypPSjeNtXGRe38xJoMrfEfehvu7X8rBSEoDLRbUb1,PEPEZO,ZOGGED PEPE BY MATT FURIE,-5.585620,0.738575,7.38575,1.061555,-0.852389


# Available Data

* the socials can be obtained from https://gmgn.ai/defi/quotation/v1/tokens/sol/EYVy8k2a3hw31NTRdQSfKke53KyQfEehRbUVA8SFpump 
* the general info stuff like burn ratios etc can also be found there, in case one were to use it for non-pump funs
* this way the rust bit is just for execution and the crucial information 
    * about mint authority
    * freeze authority 
    * liquidity pooled is still available
* it also contains a key of creator address, the previous rugs if there were any and the creator balance for the token

* it is possible to grab holders from that endpoint but that won't be backwards
  compatible since it is for real-time the moment of requesting, otherwise this
  information could be used during collection of the dataset

* https://gmgn.ai/defi/quotation/v1/tokens/top_buyers/sol/EYVy8k2a3hw31NTRdQSfKke53KyQfEehRbUVA8SFpump
  => top buyers could be useful if given wallets drive pumps, could be an
  important feature

* all of the new pump listings can be found here:
  https://gmgn.ai/defi/quotation/v1/pairs/sol/new_pairs?limit=50&orderby=open_timestamp&direction=desc&isPump=true&filters[]=not_honeypot&filters[]=pump
* interesting idea would be to grab the ones from the last hour and generally
  look for 30-50 candles when training the model

* then the **socials + top holders + creator information** could be analyzed at the time of snapshot
* the https://gmgn.ai/defi/quotation/v1/tokens/tag_wallet_count/sol/EYVy8k2a3hw31NTRdQSfKke53KyQfEehRbUVA8SFpump
  information also might be useful

* the wss://ws.gmgn.ai/stream?_t=true&tk=API_KEY (seems like) is a nice way of
  listening on new trades for a given pair there is information around realized
  profits, could be useful

In [22]:
import time
from constants import BOME

candles = watcher.get_candles(
    BOME, 0, time.time(), "4h"
)

https://gmgn.ai/defi/quotation/v1/tokens/kline/sol/ukHH6c7mMyiWCf1b9pnWe25TSpkDDt3H5pQZgZ74J82


In [24]:
# the above endpoint returns up to 1k candles
# this is good, as there is no simple way of finding the creation date without
# sending another request

# this enables to just send a request for any token that has been around for less than 16 hours
# keep the data for first say 12 hours and store that
# different intervals can be used
1000 / 60

# there could be the live stuff used (things described above)

16.666666666666668

In [23]:
len(candles.data)

569

In [78]:
from watcher import Candle

In [92]:
from watcher import last_50_candles, Timeframe

watcher = Watcher(Chain.SOL)
timeframe = Timeframe.ONE_MINUTE
start, end = last_50_candles(timeframe)
candles_response = watcher.get_candles(
    token_address="GmaPKZisgQtH1ZXx5Xx6mQD6hKHZZUKdwR9SrBdrpump",
    start=0,
    end=end,
    timeframe=timeframe.value
)
df = pd.DataFrame([vars(candle) for candle in candles_response.data])
fig = make_fig(df)
sma_period = 10
volume_spike_threshold = 1.5

# Calculate SMA of Volume
df['sma_volume'] = df['volume'].rolling(window=sma_period).mean()

# Detect Volume Spikes
df['volume_spike'] = df['volume'] > (volume_spike_threshold * df['sma_volume'])

# Detect Upward Price Movement
df['price_upward'] = df['close'] > df['close'].shift(1)

# Generate Buy Signals
df['buy_signal'] = df['volume_spike'] & df['price_upward']

# Filter buy signals
buy_signals = df[df['buy_signal']]

fig.add_trace(
    go.Scatter(
        x=df[df['buy_signal']].index,
        y=df[df['buy_signal']]['close'],
        mode='markers',
        marker=dict(
            color='red',
            size=10,
            symbol='triangle-up'
        ),
        name='Buy Signal'
    ),
    row=1,
    col=1,
)

print(len(buy_signals))
fig.show()

https://gmgn.ai/defi/quotation/v1/tokens/kline/sol/GmaPKZisgQtH1ZXx5Xx6mQD6hKHZZUKdwR9SrBdrpump
3
